<a href="https://colab.research.google.com/github/mohigeo33/lst_timeseries/blob/main/LST_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Installing and loading required libraries**



In [ ]:
# installing libraries
!pip install geemap --upgrade
!pip install geopandas
!pip install pyshp
!pip install pycrs

In [2]:
# loading necessary libraries
import os
import sys
import ee
import geemap
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
import math

**2. Initiating Google Earth Engine (GEE)**



In [3]:
# trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=BDqFA3eEwlEiYn4HWYVo7-8WAoZYK1rmSMmpLemsAG8&tc=S4sVaIoYUVjbWdok-OPjnc1f74V_k81X1LiDU9MQ6bM&cc=74QaAcdLGbc9m2RrPGAUeJTp5TwZmibfqA3_wu9SP8Q

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk4ODQuxSBtuazO8x83dGPqyjVgjg4yizWRXe0vCzF-mSwYNKebUsOY

Successfully saved authorization token.


In [4]:
# initialize the GEE.
ee.Initialize()

**3. User inputs**

In [5]:
# study period
year_start = 2000
year_end = 2021
month_start = 1
month_end = 12

# temperature threshold
t_threshold = 20

# cloud filter
max_cloud_cover = 60 # in percentage

**4. Importing the study area**

In [6]:
# mounting the google dirive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# setting the working directory
os.chdir('/content/drive/MyDrive/Thesis/Data/AOI')
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: /content/drive/MyDrive/Thesis/Data/AOI


In [ ]:
# AOI
aoi =  '/content/drive/MyDrive/Thesis/Data/AOI/aoi3.shp' # insert input here
roi = geemap.shp_to_ee(aoi)

#ploting the study area
Map = geemap.Map()
Map.addLayer(roi, {}, 'AOI')
Map.centerObject(roi, zoom=12)
Map

***Most part of the code section till calculation of LST was used from the work of (Nill et. al., 2019).***

*GEE Implementation of the Single-Channel (SC) algorithm developed by Jiménez-Muñoz & Sobrino (2003),
Jiménez-Muñoz et al. (2009) and Jiménez-Muñoz et al. (2014) for retrieving statistical metrics of LST
from Landsat TM, ETM+ and OLI-TIRS data. The atmospheric functions used in the algorithm are
approximated using data on atmospheric water vapor content from the NCEP/NCAR Reanalysis Data. Surface emissivity is calculated using the Simplified Normalized Difference Vegetation Index
Threshold (SNDVI) approach as described by Sobrino et al. (2008).*

**5. Algorithm Specifications**

In [10]:
ndvi_v = 0.86
ndvi_s = -0.64

epsilon_v = 0.985
epsilon_s = 0.97
epsilon_w = 0.99

# Jiménez‐Muñoz et al. (2009) (TM & ETM+) TIGR1761 and Jiménez‐Muñoz et al. (2014) OLI-TIRS GAPRI4838
cs_l8 = [0.04019, 0.02916, 1.01523,
         -0.38333, -1.50294, 0.20324,
         0.00918, 1.36072, -0.27514]
cs_l7 = [0.06518, 0.00683, 1.02717,
         -0.53003, -1.25866, 0.10490,
         -0.01965, 1.36947, -0.24310]
cs_l5 = [0.07518, -0.00492, 1.03189,
         -0.59600, -1.22554, 0.08104,
         -0.02767, 1.43740, -0.25844]

**6. Functions**

6.1 Rename bands and clipping

In [11]:
# cloud masking for Surface Reflectance Products (Nill et. al., 2019)
def fun_bands_l57(img):
       bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
       thermal_band = ['B6']
       new_bands = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2']
       new_thermal_bands = ['TIR']
       vnirswir = img.select(bands).multiply(0.0001).rename(new_bands)
       tir = img.select(thermal_band).multiply(0.1).rename(new_thermal_bands)
       return vnirswir.addBands(tir).copyProperties(img, ['system:time_start'])


def fun_bands_l8(img):
       bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
       thermal_band = ['B10']
       new_bands = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2']
       new_thermal_bands = ['TIR']
       vnirswir = img.select(bands).multiply(0.0001).rename(new_bands)
       tir = img.select(thermal_band).multiply(0.1).rename(new_thermal_bands)
       return vnirswir.addBands(tir).copyProperties(img, ['system:time_start'])

#clipping
def fun_clip(img):
  clip_img = img.clip(roi)
  return clip_img

6.2 Masking

In [12]:
# Function to cloud mask Landsat TM, ETM+, OLI_TIRS Surface Reflectance Products
def fun_mask_ls_sr(img):
       cloudShadowBitMask = ee.Number(2).pow(3).int()
       cloudsBitMask = ee.Number(2).pow(5).int()
       snowBitMask = ee.Number(2).pow(4).int()
       qa = img.select('pixel_qa')
       mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
              qa.bitwiseAnd(cloudsBitMask).eq(0)).And(
              qa.bitwiseAnd(snowBitMask).eq(0))
       return img.updateMask(mask)


# Function to mask LST below certain temperature threshold
def fun_mask_T(img):
    mask = img.select('LST').gt(t_threshold)
    return img.updateMask(mask)

6.3 Matching and calibration

In [13]:
# Radiometric Calibration
def fun_radcal(img):
    radiance = ee.Algorithms.Landsat.calibratedRadiance(img).rename('RADIANCE')
    return img.addBands(radiance)


# L to ee.Image
def fun_l_addband(img):
    l = ee.Image(img.get('L')).select('RADIANCE').rename('L')
    return img.addBands(l)


# Create maxDifference-filter to match TOA and SR products
maxDiffFilter = ee.Filter.maxDifference(
    difference=2 * 24 * 60 * 60 * 1000,
    leftField= 'system:time_start',
    rightField= 'system:time_start'
)

# Define join: Water vapor
join_wv = ee.Join.saveBest(
    matchKey = 'WV',
    measureKey = 'timeDiff'
)

# Define join: Radiance
join_l = ee.Join.saveBest(
    matchKey = 'L',
    measureKey = 'timeDiff'
)

6.4 Spectral Indices

In [14]:
# NDVI
def fun_ndvi(img):
    ndvi = img.normalizedDifference(['NIR', 'R']).rename('NDVI')
    return img.addBands(ndvi)

#MNDWI (Xu, 2006)
def fun_mndwi(img):
    mndwi = img.normalizedDifference(['G', 'SWIR1']).rename('MNDWI')
    return img.addBands(mndwi)

# IBI (Xu, 2008)
def fun_ibi(img):
    const = ee.Number(2)
    
    ibi = img.expression(
        '(((const * swir1) / (swir1 + nir))-((nir / (nir + red)) + (green / (green + swir1))))   / (((const * swir1) / (swir1 + nir))+((nir / (nir + red)) + (green / (green + swir1))))',           
        {
            'swir1': img.select('SWIR1'),
            'nir': img.select('NIR'),
            'red': img.select('R'),
            'green': img.select('G'),
            'const': const
        }).rename('IBI')
    return img.addBands(ibi)      

6.5 Parameter calculation

In [15]:
# Tasseled Cap Transformation (brightness, greenness, wetness) based on Christ (1985)
def fun_tcg(img):
    tcg = img.expression(
                         'B*(-0.1603) + G*(-0.2819) + R*(-0.4934) + NIR*0.7940 + SWIR1*(-0.0002) + SWIR2*(-0.1446)',
                         {
                         'B': img.select(['B']),
                         'G': img.select(['G']),
                         'R': img.select(['R']),
                         'NIR': img.select(['NIR']),
                         'SWIR1': img.select(['SWIR1']),
                         'SWIR2': img.select(['SWIR2'])
                         }).rename('TCG')
    return img.addBands(tcg)


def fun_tcb(img):
    tcb = img.expression(
                         'B*0.2043 + G*0.4158 + R*0.5524 + NIR*0.5741 + SWIR1*0.3124 + SWIR2*0.2303',
                         {
                         'B': img.select(['B']),
                         'G': img.select(['G']),
                         'R': img.select(['R']),
                         'NIR': img.select(['NIR']),
                         'SWIR1': img.select(['SWIR1']),
                         'SWIR2': img.select(['SWIR2'])
                         }).rename('TCB')
    return img.addBands(tcb)


def fun_tcw(img):
       tcw = img.expression(
              'B*0.0315 + G*0.2021 + R*0.3102 + NIR*0.1594 + SWIR1*(-0.6806) + SWIR2*(-0.6109)',
              {
                     'B': img.select(['B']),
                     'G': img.select(['G']),
                     'R': img.select(['R']),
                     'NIR': img.select(['NIR']),
                     'SWIR1': img.select(['SWIR1']),
                     'SWIR2': img.select(['SWIR2'])
              }).rename('TCW')
       return img.addBands(tcw)


# Fraction Vegetation Cover (FVC)
def fun_fvc(img):
    fvc = img.expression(
        '((NDVI-NDVI_s)/(NDVI_v-NDVI_s))**2',
        {
            'NDVI': img.select('NDVI'),
            'NDVI_s': ndvi_s,
            'NDVI_v': ndvi_v
        }
    ).rename('FVC')
    return img.addBands(fvc)


# Scale Emissivity (Epsilon) between NDVI_s and NDVI_v
def fun_epsilon_scale(img):
    epsilon_scale = img.expression(
        'epsilon_s+(epsilon_v-epsilon_s)*FVC',
        {
            'FVC': img.select('FVC'),
            'epsilon_s': epsilon_s,
            'epsilon_v': epsilon_v
        }
    ).rename('EPSILON_SCALE')
    return img.addBands(epsilon_scale)


# Emissivity (Epsilon)
def fun_epsilon(img):
    pseudo = img.select(['NDVI']).set('system:time_start', img.get('system:time_start'))
    epsilon = pseudo.where(img.expression('NDVI > NDVI_v',
                                          {'NDVI': img.select('NDVI'),
                                           'NDVI_v': ndvi_v}), epsilon_v)
    epsilon = epsilon.where(img.expression('NDVI < NDVI_s && NDVI >= 0',
                                           {'NDVI': img.select('NDVI'),
                                            'NDVI_s': ndvi_s}), epsilon_s)
    epsilon = epsilon.where(img.expression('NDVI < 0',
                                           {'NDVI': img.select('NDVI')}), epsilon_w)
    epsilon = epsilon.where(img.expression('NDVI <= NDVI_v && NDVI >= NDVI_s',
                                           {'NDVI': img.select('NDVI'),
                                            'NDVI_v': ndvi_v,
                                            'NDVI_s': ndvi_s}), img.select('EPSILON_SCALE')).rename('EPSILON')
    return img.addBands(epsilon)


# Function to scale WV content product
def fun_wv_scale(img):
    wv_scaled = ee.Image(img.get('WV')).multiply(0.1).rename('WV_SCALED')
    wv_scaled = wv_scaled.resample('bilinear')
    return img.addBands(wv_scaled)

6.6 Land surface temperature calculation

In [16]:
# Atmospheric Functions
def fun_af1(cs):
    def wrap(img):
        af1 = img.expression(
            '('+str(cs[0])+'*(WV**2))+('+str(cs[1])+'*WV)+('+str(cs[2])+')',
            {
                'WV': img.select('WV_SCALED')
            }
        ).rename('AF1')
        return img.addBands(af1)
    return wrap


def fun_af2(cs):
    def wrap(img):
        af2 = img.expression(
            '('+str(cs[3])+'*(WV**2))+('+str(cs[4])+'*WV)+('+str(cs[5])+')',
            {
                'WV': img.select('WV_SCALED')
            }
        ).rename('AF2')
        return img.addBands(af2)
    return wrap


def fun_af3(cs):
    def wrap(img):
        af3 = img.expression(
            '('+str(cs[6])+'*(WV**2))+('+str(cs[7])+'*WV)+('+str(cs[8])+')',
            {
                'WV': img.select('WV_SCALED')
            }
        ).rename('AF3')
        return img.addBands(af3)
    return wrap


# Gamma Functions
def fun_gamma_l8(img):
    gamma = img.expression('(BT**2)/(1324*L)',
                           {'BT': img.select('TIR'),
                            'L': img.select('L')
                            }).rename('GAMMA')
    return img.addBands(gamma)


def fun_gamma_l7(img):
    gamma = img.expression('(BT**2)/(1277*L)',
                           {'BT': img.select('TIR'),
                            'L': img.select('L')
                            }).rename('GAMMA')
    return img.addBands(gamma)


def fun_gamma_l5(img):
    gamma = img.expression('(BT**2)/(1256*L)',
                           {'BT': img.select('TIR'),
                            'L': img.select('L')
                            }).rename('GAMMA')
    return img.addBands(gamma)


# Delta Functions
def fun_delta_l8(img):
    delta = img.expression('BT-((BT**2)/1324)',
                           {'BT': img.select('TIR')
                            }).rename('DELTA')
    return img.addBands(delta)


def fun_delta_l7(img):
    delta = img.expression('BT-((BT**2)/1277)',
                           {'BT': img.select('TIR')
                            }).rename('DELTA')
    return img.addBands(delta)


def fun_delta_l5(img):
    delta = img.expression('BT-((BT**2)/1256)',
                           {'BT': img.select('TIR')
                            }).rename('DELTA')
    return img.addBands(delta)


# Land Surface Temperature
def fun_lst(img):
    lst = img.expression(
        '(GAMMA*(((1/EPSILON)*(AF1*L+AF2))+AF3)+DELTA)-273.15',
        {
            'GAMMA': img.select('GAMMA'),
            'DELTA': img.select('DELTA'),
            'EPSILON': img.select('EPSILON'),
            'AF1': img.select('AF1'),
            'AF2': img.select('AF2'),
            'AF3': img.select('AF3'),
            'L': img.select('L')
        }
    ).rename('LST')
    return img.addBands(lst)


def fun_mask_lst(img):
    mask = img.select('LST').gt(t_threshold)
    return img.updateMask(mask)

6.7 Descriptive statistics extraction and date variables

In [17]:
#function for extracting image info
def fctn_get_image_stats(img):
  img_lst = img.select('LST')
  img_ndvi = img.select('NDVI')
  img_mndwi = img.select('MNDWI')
  img_ibi = img.select('IBI')
  # img_cloud_cover = img.get('CLOUD_COVER').getInfo()
  

  img_lst_mean_value = img_lst.reduceRegion(ee.Reducer.mean(), roi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['LST']
  img_ndvi_mean_value = img_ndvi.reduceRegion(ee.Reducer.mean(), roi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['NDVI']
  img_mndwi_mean_value = img_mndwi.reduceRegion(ee.Reducer.mean(), roi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['MNDWI']
  img_ibi_mean_value = img_ibi.reduceRegion(ee.Reducer.mean(), roi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['IBI']


  img_lst_max_value = img_lst.reduceRegion(ee.Reducer.max(), roi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['LST']
  img_ndvi_max_value = img_ndvi.reduceRegion(ee.Reducer.max(), roi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['NDVI']
  img_mndwi_max_value = img_mndwi.reduceRegion(ee.Reducer.max(), roi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['MNDWI']
  img_ibi_max_value = img_ibi.reduceRegion(ee.Reducer.max(), roi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['IBI']


  img_lst_min_value = img_lst.reduceRegion(ee.Reducer.min(), roi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['LST']
  img_ndvi_min_value = img_ndvi.reduceRegion(ee.Reducer.min(), roi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['NDVI']
  img_mndwi_min_value = img_mndwi.reduceRegion(ee.Reducer.min(), roi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['MNDWI']
  img_ibi_min_value = img_ibi.reduceRegion(ee.Reducer.min(), roi, 30,  crs = 'EPSG:32648', bestEffort = True, maxPixels = 1e9).getInfo()['IBI']

  
  img_date = img.date().getInfo()['value']
  
  img_sytemindex = img.get('system:index').getInfo()

  img_cloud_cover = img.get('L')
  img_cloud_cover = ee.Image(img_cloud_cover)
  img_cloud_cover = img_cloud_cover.get('CLOUD_COVER').getInfo()
  # print(img_cloud_cover)

  img_all_info = {
      'system:index': img_sytemindex,
      'date': img_date,
      'mean_lst' : img_lst_mean_value,
      'mean_ndvi' : img_ndvi_mean_value,
      'mean_mndwi' : img_mndwi_mean_value,
      'mean_ibi' : img_ibi_mean_value,

      'min_lst' : img_lst_min_value,
      'min_ndvi' : img_ndvi_min_value,
      'min_mndwi' : img_mndwi_min_value,
      'min_ibi' : img_ibi_min_value,
      
      'max_lst' : img_lst_max_value,
      'max_ndvi' : img_ndvi_max_value,
      'max_mndwi' : img_mndwi_max_value,
      'max_ibi' : img_ibi_max_value,
      'cloud_cover': img_cloud_cover
      
  }

  return img_all_info

# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['Millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

**7. Calling the image collections**

In [ ]:
# Landsat 5 TM
imgCol_L5_TOA = ee.ImageCollection('LANDSAT/LT05/C01/T1')\
    .filterBounds(roi)\
    .filter(ee.Filter.calendarRange(year_start,year_end,'year'))\
    .filter(ee.Filter.calendarRange(month_start,month_end,'month'))\
    .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover))\
    .select(['B6'])

imgCol_L5_SR = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')\
    .filterBounds(roi)\
    .filter(ee.Filter.calendarRange(year_start,year_end,'year'))\
    .filter(ee.Filter.calendarRange(month_start,month_end,'month'))\
    .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover))\
    .map(fun_mask_ls_sr)

imgCol_L5_SR = imgCol_L5_SR.map(fun_bands_l57)

# Landsat 7 ETM+
imgCol_L7_TOA = ee.ImageCollection('LANDSAT/LE07/C01/T1')\
    .filterBounds(roi)\
    .filter(ee.Filter.calendarRange(year_start,year_end,'year'))\
    .filter(ee.Filter.calendarRange(month_start,month_end,'month'))\
    .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover))\
    .select(['B6_VCID_2'])

imgCol_L7_SR = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
    .filterBounds(roi)\
    .filter(ee.Filter.calendarRange(year_start,year_end,'year'))\
    .filter(ee.Filter.calendarRange(month_start,month_end,'month'))\
    .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover))\
    .map(fun_mask_ls_sr)

imgCol_L7_SR = imgCol_L7_SR.map(fun_bands_l57)

# Landsat 8 OLI-TIRS
imgCol_L8_TOA = ee.ImageCollection('LANDSAT/LC08/C01/T1')\
    .filterBounds(roi)\
    .filter(ee.Filter.calendarRange(year_start,year_end,'year'))\
    .filter(ee.Filter.calendarRange(month_start,month_end,'month'))\
    .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover))\
    .select(['B10'])

imgCol_L8_SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterBounds(roi)\
    .filter(ee.Filter.calendarRange(year_start,year_end,'year'))\
    .filter(ee.Filter.calendarRange(month_start,month_end,'month'))\
    .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_cover))\
    .map(fun_mask_ls_sr)

imgCol_L8_SR = imgCol_L8_SR.map(fun_bands_l8)

# NCEP/NCAR Water Vapor Product
imgCol_WV = ee.ImageCollection('NCEP_RE/surface_wv')\
    .filterBounds(roi)\
    .filter(ee.Filter.calendarRange(year_start,year_end,'year'))\
    .filter(ee.Filter.calendarRange(month_start,month_end,'month'))

print("Total LS 5 collection TOA: ",imgCol_L5_TOA.size().getInfo())
print("Total LS 5 collection SR: ",imgCol_L5_SR.size().getInfo())
print("Total LS 7 collection TOA: ",imgCol_L7_TOA.size().getInfo())
print("Total LS 7 collection SR: ",imgCol_L7_SR.size().getInfo())
print("Total LS 8 collection TOA: ",imgCol_L8_TOA.size().getInfo())
print("Total LS 8 collection SR: ",imgCol_L8_SR.size().getInfo())    

**8. Calculation of LST and other indices**

In [19]:
# TOA (Radiance) and SR
imgCol_L5_TOA = imgCol_L5_TOA.map(fun_radcal)
imgCol_L7_TOA = imgCol_L7_TOA.map(fun_radcal)
imgCol_L8_TOA = imgCol_L8_TOA.map(fun_radcal)

imgCol_L5_SR = ee.ImageCollection(join_l.apply(imgCol_L5_SR, imgCol_L5_TOA, maxDiffFilter))
imgCol_L7_SR = ee.ImageCollection(join_l.apply(imgCol_L7_SR, imgCol_L7_TOA, maxDiffFilter))
imgCol_L8_SR = ee.ImageCollection(join_l.apply(imgCol_L8_SR, imgCol_L8_TOA, maxDiffFilter))

imgCol_L5_SR = imgCol_L5_SR.map(fun_l_addband)
imgCol_L7_SR = imgCol_L7_SR.map(fun_l_addband)
imgCol_L8_SR = imgCol_L8_SR.map(fun_l_addband)

# Water Vapor
imgCol_L5_SR = ee.ImageCollection(join_wv.apply(imgCol_L5_SR, imgCol_WV, maxDiffFilter))
imgCol_L7_SR = ee.ImageCollection(join_wv.apply(imgCol_L7_SR, imgCol_WV, maxDiffFilter))
imgCol_L8_SR = ee.ImageCollection(join_wv.apply(imgCol_L8_SR, imgCol_WV, maxDiffFilter))

imgCol_L5_SR = imgCol_L5_SR.map(fun_wv_scale)
imgCol_L7_SR = imgCol_L7_SR.map(fun_wv_scale)
imgCol_L8_SR = imgCol_L8_SR.map(fun_wv_scale)

# Atmospheric Functions
imgCol_L5_SR = imgCol_L5_SR.map(fun_af1(cs_l5))
imgCol_L5_SR = imgCol_L5_SR.map(fun_af2(cs_l5))
imgCol_L5_SR = imgCol_L5_SR.map(fun_af3(cs_l5))

imgCol_L7_SR = imgCol_L7_SR.map(fun_af1(cs_l7))
imgCol_L7_SR = imgCol_L7_SR.map(fun_af2(cs_l7))
imgCol_L7_SR = imgCol_L7_SR.map(fun_af3(cs_l7))

imgCol_L8_SR = imgCol_L8_SR.map(fun_af1(cs_l8))
imgCol_L8_SR = imgCol_L8_SR.map(fun_af2(cs_l8))
imgCol_L8_SR = imgCol_L8_SR.map(fun_af3(cs_l8))

# Delta and Gamma Functions
imgCol_L5_SR = imgCol_L5_SR.map(fun_delta_l5)
imgCol_L7_SR = imgCol_L7_SR.map(fun_delta_l7)
imgCol_L8_SR = imgCol_L8_SR.map(fun_delta_l8)

imgCol_L5_SR = imgCol_L5_SR.map(fun_gamma_l5)
imgCol_L7_SR = imgCol_L7_SR.map(fun_gamma_l7)
imgCol_L8_SR = imgCol_L8_SR.map(fun_gamma_l8)

# Merge Collections
imgCol_merge = imgCol_L8_SR.merge(imgCol_L7_SR).merge(imgCol_L5_SR)
imgCol_merge = imgCol_merge.sort('system:time_start')

# Clipping to ROI
imgCol_merge = imgCol_merge.map(fun_clip)

# Parameters and Indices
imgCol_merge = imgCol_merge.map(fun_ndvi)
imgCol_merge = imgCol_merge.map(fun_ibi)
imgCol_merge = imgCol_merge.map(fun_mndwi)
imgCol_merge = imgCol_merge.map(fun_tcg)
imgCol_merge = imgCol_merge.map(fun_tcb)
imgCol_merge = imgCol_merge.map(fun_tcw)

imgCol_merge = imgCol_merge.map(fun_fvc)
imgCol_merge = imgCol_merge.map(fun_epsilon_scale)
imgCol_merge = imgCol_merge.map(fun_epsilon)


# LST
imgCol_merge = imgCol_merge.map(fun_lst)
imgCol_merge = imgCol_merge.map(fun_mask_lst)



In [ ]:
imgCol_merge

In [21]:
print("Total image collection merged and clipped: ",imgCol_merge.size().getInfo())

Total image collection merged and clipped:  462


In [ ]:
#testing LST output
img1 = imgCol_merge.sort('system:time_start', False).first()
img1 = img1.select('LST')
lst_max = ee.Number(img1.reduceRegion(**{
  "reducer": ee.Reducer.max(),
  "scale": 30,
  "maxPixels": 10**9
}).values().get(0));
print( 'max value of lst',lst_max.getInfo())

lst_min = ee.Number(img1.reduceRegion(**{
  "reducer": ee.Reducer.min(),
  "scale": 30,
  "maxPixels": 10**9
}).values().get(0));
print( 'min value of lst',lst_min.getInfo())

**9. Descriptive statistics extraction**

In [ ]:
# final merged image collection
imgCol_merge

# converting image collection to list
doi = imgCol_merge
doiList = doi.toList(doi.size())
doiList_size = doiList.size().getInfo()
print('Total Images in Data of Interest (doi) dataset: ',doiList_size)

# creating the dataframe
df = pd.DataFrame(columns=['SystemIndex', 'Millis', 'MeanIBI','MaxIBI','MinIBI', 'MeanNDVI','MaxNDVI','MinNDVI', 'MeanMNDWI', 'MaxMNDWI', 'MinMNDWI', 'MeanLST', 'MaxLST', 'MinLST', 'Cloud'])
# iteration
for i in tqdm(range(doiList_size)):
  image = ee.Image(doiList.get(i))
  image_info = fctn_get_image_stats(image)
  
  df = df.append({'SystemIndex': image_info['system:index'], 
                  'Millis':  image_info['date'], 
                  'MeanIBI': image_info['mean_ibi'], 
                  'MaxIBI': image_info['max_ibi'], 
                  'MinIBI': image_info['min_ibi'],
                  'MeanNDVI': image_info['mean_ndvi'], 
                  'MaxNDVI': image_info['max_ndvi'], 
                  'MinNDVI': image_info['min_ndvi'],
                  'MeanMNDWI': image_info['mean_mndwi'], 
                  'MaxMNDWI': image_info['max_mndwi'], 
                  'MinMNDWI': image_info['min_mndwi'],
                  'MeanLST': image_info['mean_lst'], 
                  'MaxLST': image_info['max_lst'], 
                  'MinLST': image_info['min_lst'],
                  'Cloud' : image_info['cloud_cover']
                  }, ignore_index=True)

In [ ]:
df

In [ ]:
#adding the date variables
df2 = add_date_info(df)

# dataframe with date variable
df2

In [ ]:
# export the dataframe to a CSV file
df2.to_csv('dfall.csv', index=False)

**The produced dataframe named 'df' is analysed in further statistical analysis**